# Prompt Engineering: Use OpenAI to Analyze Twitter Data 
This is a simple tutorial teaching prompt engineering basics and analyzing Twitter data with OpenAI large language models (LLM).
Please purchase an [OpenAI API](https://openai.com/index/openai-api/) and store it in a safe place. This tutorial uses [AWS Secretes Manager](https://aws.amazon.com/secrets-manager/) to store the API keys.  

## Large Language Model Basics
LLM repeatable predicts the next world using supervised learning. To predict the following sentence: 

`Learning data science in the cloud with AI`

A model needs to learn to predict the following steps:

|Input|Output|
|:---|---|
|Learning data science |in |
|Learning data science in |the | 
|Learning data science in the |cloud |
|Learning data science in the cloud |with |
|Learning data science in the cloud with |AI|

To train an LLM model:
1. Training a base LLM model on a large amount of training data to predict the next word 
2. Fine-tune on examples where outputs follow instructions in the input 
3. Human rates quality of different LLM outputs 
4. Tune LLM to generate outputs with higher rates using RLHF (Reinforcement learning from human feedback)

## Set up OpenAI Models

Load the API keys with AWS Secrets Manage Function 

In [1]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Install Python libraries.

- pymongo: manage the MongoDB database
- openai: call the OpenAI APIs.

In [2]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 38.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Load the OpenAI API key and define a `openai_help` function.

In [4]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

Temperature: 
- Low temperature: always choose the most likely response, reliable, predictable responses  
- High temperature: diverse responses, more creative responses

Tokens and Models: 
- LLM predicts tokens, which are commonly occurring sequences of characters. 
- One token is about four characters in English, and 100 tokens are roughly 75 words. Check [token estimate](https://platform.openai.com/tokenizer).
- Different models can process various amounts of tokens at different performance levels and costs. Check [OpenAI models](https://platform.openai.com/docs/models) for more details.

Roles:
- system: specify the overall tone or behavior of the assistant 
- user: instruction given to the LLM
- assistant: LLM responded content, we also can provide content in few-shot promoting or histories of conversations


A simple example using [gtp-4o](https://platform.openai.com/docs/models/gpt-4o) and temperature 0.

In [5]:
messages = [{"role": "user", "content": "What is the capital of USA"}]

print(openai_help(messages))

The capital of the United States is Washington, D.C.


Add a system message asking LLM to act as a high school teacher with different temperatures.

In [6]:
messages = [
    {"role": "system", "content": "use tone as a high school teacher"},
    {"role": "user", "content": "What is the capital of USA"}
    ]

print(openai_help(messages, temperature = 0.8))

The capital of the United States is Washington, D.C. This city is home to all three branches of the federal government and is a very important place for the country's political activities. If you have more questions about Washington, D.C., or anything else related to U.S. geography, feel free to ask!


Add assistant messages to teach LLM what `##` is.

In [7]:
messages = [
    {"role": "user", "content": "What is 1##1"},
    {"role": "assistant", "content": "it is 11"},
    {"role": "user", "content": "What is 2##2"},
    {"role": "assistant", "content": "it is 22"},
    {"role": "user", "content": "What is 3##3"},
    ]
print(openai_help(messages))

It is 33.


## Prompt Engineering Principles 
- Use delimiters to separate different parts of a prompt to provide clear instructions and prevent prompt injections.
- Structure outputs in JSON documents or other formats to use the outputs in subsequent steps 
- Few-shot promoting: provide successful examples of a task and then ask the model to perform a similar task. 
- Chain of thought reasoning: request a series of reasoning steps in prompts to help the model achieve correct answers
- Chain of prompts: split a task into multiple prompts where each prompt can focus on a sub-task at a time and take different actions at different stages. It saves tokens, is easier to test, can involve human input, or use external tools.
- Interactive process 
  1. Try something first 
  2. Analyses the result, identify errors, and redefine the prompt 
  3. Test the prompts with different datasets 


An example using delimiters, structured output and few-shot promoting:

In [8]:
delimiter = '###'
sentence1 = 'I love cat.'
sentence2 = 'I love dog.'
messages = [
    {"role": "system", "content": f"""analyze the sentiment in a sentence delimitered by {delimiter},
                                     return the result as a JSON document"""},
    {"role": "user", "content": f"{delimiter}{sentence1}{delimiter}"},
    {"role": "assistant", "content": "{sentiment:positive}"},
    {"role": "user", "content": f"{delimiter}{sentence2}{delimiter}"}
    ]

print(openai_help(messages))

{ "sentiment": "positive" }


## Analyze Twitter data

### Connect to the MongoDB cluster

In [9]:
import pymongo
from pymongo import MongoClient
mongodb_connect = get_secret('mongodb')['connection_string']

mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
tweet_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'tweet.id_1'

### Extract Tweets

In [19]:
filter={

    
}
project={
    'tweet.text': 1, 
    'tweet.id': 1
}
#rename the client to mongo_client
result = mongo_client['demo']['tweet_collection'].find(
  filter=filter,
  projection=project
)

In [20]:
tweet_data = []
for tweet in result:
    tweet_data.append(tweet['tweet']['text'])

In [21]:
print('Number of tweets: ',len(tweet_data))

Number of tweets:  298


### Summarization 
- Analyze election tweets with delimiters 
- Change the size of the summarization 
- Summarize tweets and focus on different perspectives. 

In [22]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter}"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets cover a wide range of topics, primarily focusing on the upcoming 2024 U.S. presidential election and related political issues. There are discussions about election integrity, accusations of election interference, and debates over voter ID laws. Some tweets express concerns about media bias and misinformation, while others discuss the potential impact of Donald Trump's candidacy. Additionally, there are mentions of international issues, such as accusations against Binance for facilitating terrorism financing, and discussions about the role of various countries in supporting terrorism. The tweets also touch on domestic political dynamics, including the influence of dark money in elections and the role of key political figures like Kamala Harris and Joe Biden.


In [23]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    limit the summary to 20 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets discuss election interference, voter ID laws, media bias, and allegations of terrorism financing, with a focus on the 2024 U.S. election.


In [24]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    focus on how people discuss AI,
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets discuss AI in the context of election predictions and market speculation. Some users mention AI's role in analyzing election outcomes, while others highlight its use in trading platforms like the Trump Perpetual Prediction Market. The conversation reflects a mix of skepticism and interest in AI's influence on political forecasting.


### Moderation 
- Iterate each tweet and use the [moeration endpoint](https://platform.openai.com/docs/api-reference/moderations) to identify flagged tweets
- Print flagged tweets


In [25]:
def flag_help(tweet):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=tweet)

    if response.results[0].flagged:
        print('===')
        cat_dict = response.results[0].categories.to_dict()
        for cat in cat_dict.keys():
            if cat_dict.get(cat):
                print (cat)
                print(tweet)

In [26]:
for tweet in tweet_data:
    flag_help(tweet)

===
violence
RT @ecomarxi: “There’s an election in three weeks. We might lose because we’re committing genocide.”

“I know! Let’s promise to do what we…
===
harassment
@LauraLoomer Their time will be coming to an end soon once Trump wins this election. They will be running and hiding. No more protection from this treasonous administration.
===
harassment
RT @TheRickyDavila: Corrupt GOP scumbag James Comer basically went on propaganda outlet newsmax to say the biggest irregularity in the 2020…
===
harassment
@PettyLupone @Bobbysworld2021 You people are liars and pathetic human beings. There’s a reason you’re going to lose this election and the next one and the next one. You’re weak humans that no sane person wants to follow.
===
violence
RT @democracynow: Uncommitted Co-Founder Abbas Alawieh on U.S. Election &amp; Family in Lebanon Fleeing Israeli Bombs https://t.co/f00eSotc84
===
violence
RT @nypost: Israel is ready to strike Iran — with attack expected before US election: report https

### Transforming
- Translating to a different language 
- Transform tones, such as formal vs. informal.  


In [27]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into Chinese"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

@C_Farthammer 那个胖胖的小精灵在2020年大选结果确定之前就宣布了亚利桑那州。福克斯新闻和CNN没什么区别。
RT @MargieVotes: #DemsUnited #ProudBlue 
随着我们接近2024年选举，我们必须发声。

我知道我们很累。选举还有几周就要到了……
@NickGarzilli @BruneElections @SMCosta6 @tencor_7144 波多黎各只是为了进行人工计票。在选举之夜，投票站工作人员会统计州长、参议员、州代表、市长和镇议会代表的选票。平均来说，有120万张选票。现在，他们已经转向电子（Dominion）计票，并出现了计票问题。
RT @PATRICKHENRY97: 那么，为什么自由派不想要投票身份证？哦，我知道他们永远不会再赢得选举。身份证和一天投票。L…
RT @RealMattCouch: 这太疯狂了。这是媒体公然的选举干预，他们必须被特朗普的司法部起诉。
RT @OccupyDemocrats: 突发新闻：唐纳德·特朗普在荒谬地声称权力和平移交后，被一名采访者羞辱…
RT @Indian_Analyzer: 选举专员Rajiv Kumar：
“在投票前6天，新的电池被插入电子投票机。即使是电池也必须……”
选举伎俩 🤷🏾‍♂️ https://t.co/60VhB47cXA
RT @ecomarxi: “我们一生中最重要的选举” https://t.co/zQDQ2Dp42F
抱歉，我无法协助翻译该内容。
RT @newrepublic: 泄露的电子邮件揭示了Rasmussen Reports的真相，以及特朗普竞选团队如何违反选举法。

https://t…
RT @BrandtRobinson: @TheClassicPhil 问题在于你正在投票给一个试图推翻我们总统选举结果的人…
RT @ecomarxi: “三周后有选举。我们可能会输，因为我们正在犯下种族灭绝。”

“我知道！让我们承诺去做我们……”
RT @MJTruthUltra: 他们又试图窃取乔治亚州的选举……腐败的乔治亚州法官称官员必须认证选举结果……
RT @BradBeauregardJ: 这位来自宾夕法尼亚州的86岁老人说，2024年的这次选举是他一生中最重要的选举…
RT @Maserati1123: @Accou

KeyboardInterrupt: 

In [28]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimited by {delimiter} in the tone like Stewie """},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

Oh, how delightful! The rotund little leprechaun made his call on Arizona before the ink was even dry on the ballots. Fox News, you say? Just as insufferable as CNN, it seems. What a charming circus of incompetence!
RT @MargieVotes: #DemsUnited #ProudBlue 
Ah, the 2024 election looms ever closer, doesn't it? We mustn't let fatigue silence us. Rally, my dear compatriots, for the final stretch is upon us...
Oh, splendid! Puerto Rico, once a bastion of meticulous hand-counting, now entrusts its electoral fate to the whims of electronic contraptions. Dominion, you say? How delightfully modern! I do hope they’ve accounted for the inevitable counting conundrums. One can only imagine the chaos that might ensue. Do keep me posted, won’t you?
Ah, yes, the age-old conundrum of voter IDs. One might ponder why those liberal-minded folks are so averse to such a notion. Could it be that without them, their electoral prospects would dwindle faster than Brian's attention span? IDs and a single day of 

KeyboardInterrupt: 

### Inferring
- Use step-by-step instructions with delimiters to:
  1. Identify sentiments
  2. Identify emotions
  3. Extract mentioned people's names
  3. Identify whether a tweet supports Democratic, Republican, or unknown 
  4. Extract outputs into a structured JSON document. 
- Identify topics from Tweets. 


In [29]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a single word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    print(openai_help(messages))

{
  "sentiment": "negative",
  "emotion": "frustration",
  "mentioned": ["@C_Farthammer"],
  "support": "Republican"
}
{
  "sentiment": "neutral",
  "emotion": "determination",
  "mentioned": ["MargieVotes"],
  "support": "Democratic"
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": [
    "NickGarzilli",
    "BruneElections",
    "SMCosta6",
    "tencor_7144"
  ],
  "support": "neutral"
}
{
  "sentiment": "negative",
  "emotion": "skepticism",
  "mentioned": ["PATRICKHENRY97"],
  "support": "Republican"
}
{
  "sentiment": "negative",
  "emotion": "outrage",
  "mentioned": ["RealMattCouch", "Trump"],
  "support": "Republican"
}
{
  "sentiment": "negative",
  "emotion": "humiliation",
  "mentioned": ["Donald Trump"],
  "support": "Democratic"
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": ["Rajiv Kumar"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "indifference",
  "mentioned": [],
  "support": "neutral"
}
{
  "sent

KeyboardInterrupt: 

In [30]:

messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} to identify 10 topics, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter} "}]
print(openai_help(messages))

{
  "topics": [
    "Election Integrity and Fraud Allegations",
    "Media and Election Interference",
    "Voter ID and Voting Laws",
    "Election Predictions and Polls",
    "Political Campaigns and Strategies",
    "Election Security and Technology",
    "Political Figures and Their Influence",
    "Election Financing and Dark Money",
    "International Influence and Election Interference",
    "Terrorism Financing and Cryptocurrency"
  ]
}


### Expanding with multiple prompts 
- Identify which party receives majority supports
- Provide contexts in the system message
- Create a chatbot to answer users’ inquiry  


In [ ]:
analysis_result = []
from tqdm import tqdm
for tweet in tqdm(tweet_data):
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a singple word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    analysis_result.append(openai_help(messages))


 14%|█▍        | 42/298 [00:43<04:20,  1.02s/it]

In [ ]:
print(analysis_result)

In [ ]:
messages = [
        {"role": "system", "content": f"""analyze the tweet analysis reuslt delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} count the number of tweets that support Democratic and Republican;
                                        step 2 {delimiter} identify the common sentiments and emotoions to each mentioned people;
                                        step 3 {delimiter} organize the result in a json document with keys <Democratic count>, <Republican count>, <people name>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{analysis_result}{delimiter} "}]
analysis_summary = openai_help(messages)
print(analysis_summary)

## Create a chatbot

In [ ]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

chat_history = [

{"role": "system", "content": f"""you are a chabot answer user questions based on the tweets,
                                {delimiter}{tweet_data}{delimiter}, 
                                if user mentioned a people name in the {delimiter}{analysis_summary}{delimiter} people field,report the corresponding sentiment and emotion,
                            
                            """}
]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    print(f"Chatbot: {reply}")

## Reference
- Isa Fulford and Andrew Ng. n.d.-a. *“Building Systems with the ChatGPT API.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/.
- ———. n.d.-b. *“ChatGPT Prompt Engineering for Developers.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/.
- OpenAI. n.d. *“OpenAI Documents.”* OpenAI. Accessed October 18, 2024. https://platform.openai.com.
